In [2]:
# import sys
import xml.etree.ElementTree as ET
import re
from tqdm import tqdm
from nltk.stem import PorterStemmer

import gensim

args = ['./data/pass_full.queries', 'trec678.vec.txt', 'TREC_DL.NN']
if len(args) < 3:
    # print('Needs 2 arguments - \n1. TREC original query file (.xml format)\n'
    #       '2. word2vec file (.txt)\n'
    #       '3. <query-term \t NN-list> output file path')
    print('Needs 3 arguments - \n1. TREC original query file (a txt file)\n'
          '2. word2vec file (.txt)\n'
          '3. <query-term \t NN-list> output file path')
#     exit(0)

arg_trec_query_file = args[0]
arg_w2v_file = args[1]
arg_nn_out_file = args[2]

fw_NN_terms = open(f'./output/{arg_nn_out_file}', 'w+')


In [3]:
# read stop words
stopwords = []
try:
    file = open('../resources/smart-stopwords', 'r', encoding='utf-8')

except FileNotFoundError:
    print(f"Results about stopwords are not found.")

with file:
    
    Lines = file.readlines()
        
    for line in Lines:
        stopwords.append(line[:-1])
    file.close()
stopwords = set(stopwords)
print(stopwords)

{'theres', 'insofar', 'they', 'z', 'sent', 'do', 'came', 'specified', 'her', 'looking', 'regards', 'where', 'sometime', 'particular', 'now', 'seemed', 'r', 'when', 'que', 'mean', "they've", 'the', 'say', "i'm", 'onto', "aren't", "didn't", 'own', 'afterwards', 'at', 'other', 'believe', 'eight', 'under', 'together', 'ought', "we've", "we're", 'nevertheless', 'rather', 'p', "a's", 'tried', 'whatever', 'f', 'gotten', 'thorough', 'necessary', 'up', 'etc', 'for', 'concerning', 'somewhere', 'well', 'yes', 'especially', 'ask', 'placed', 'ignored', "couldn't", 'herself', 'saying', 'perhaps', 'become', 'sure', 'quite', 'somewhat', 'from', 'respectively', 'most', 'e', "they'll", 'sup', 'already', 'downwards', 'b', 'myself', 'becomes', 'currently', 'various', 'contain', 'does', 'anybody', 'former', 'took', 'ours', 'two', 'inner', 'better', 'whose', 'within', 'greetings', 'therefore', 'howbeit', 'only', 'each', 'appreciate', 'six', "wasn't", 'by', 'sorry', 'those', 'seeming', 'everywhere', "ain't",

In [4]:
# start ...
original_query_list = []
qterm_NN_dict = {}

stemmer = PorterStemmer()

try:
    file = open(arg_trec_query_file, 'r', encoding='utf-8')

except FileNotFoundError:
    print(f"Results about {arg_trec_query_file} are not found.")

with file:
    
    Lines = file.readlines()
        
    for line in Lines:
        query = line.split('\t')[1][:-1].lower().strip()
        for term in query.split(' '):
            original_query_list.append(stemmer.stem(term))    
    file.close()
print(original_query_list)


['anthropolog', 'definit', 'of', 'environ', 'axon', 'termin', 'or', 'synapt', 'knob', 'definit', 'caus', 'of', 'left', 'ventricular', 'hypertrophi', 'caus', 'of', 'militari', 'suicid', 'cost', 'of', 'interior', 'concret', 'floor', 'definit', 'declaratori', 'judgment', 'definit', 'of', 'a', 'sigmet', 'differ', 'between', 'a', 'mcdoubl', 'and', 'a', 'doubl', 'cheeseburg', 'differ', 'between', 'rn', 'and', 'bsn', 'do', 'goldfish', 'grow', 'doe', 'legionella', 'pneumophila', 'caus', 'pneumonia', 'exampl', 'of', 'monoton', 'function', 'exon', 'definit', 'biolog', 'how', 'are', 'some', 'shark', 'warm', 'blood', 'how', 'long', 'is', 'life', 'cycl', 'of', 'flea', 'how', 'to', 'find', 'the', 'midseg', 'of', 'a', 'trapezoid', 'is', 'cdg', 'airport', 'in', 'main', 'pari', 'lp', 'law', 'definit', "medicare'", 'definit', 'of', 'mechan', 'ventil', 'right', 'pelvic', 'pain', 'caus', 'rsa', 'definit', 'key', 'type', 'of', 'dysarthria', 'from', 'cerebr', 'palsi', 'what', 'are', 'the', 'social', 'determ

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format(arg_w2v_file, binary=False)


In [6]:
for term in tqdm(original_query_list):
    # if term not in ['am', '103', 'three', 'u.s.']:
        print('query term : ', term)
        try:
            vector = model.most_similar(positive=[term], topn=5)
        except KeyError:
            continue
        qterm_NN_dict[term] = vector
        print(vector)
        nn_list = []
        for nn_term in vector:
            nn_list.append(nn_term[0])
        qterm_NN_dict[term] = nn_list
print('dictionary ready...')



  0%|                                                                                                                                                                                                               | 0/558 [00:00<?, ?it/s]

query term :  anthropolog


  1%|█▍                                                                                                                                                                                                     | 4/558 [00:00<01:47,  5.16it/s]

[('ethnolog', 0.747401773929596), ('anthropologist', 0.7329198122024536), ('ethnograph', 0.6933757662773132), ('zoologist', 0.6804481744766235), ('wolpert', 0.6775830984115601)]
query term :  definit
[('mean', 0.842530369758606), ('defin', 0.8261494040489197), ('appli', 0.8240614533424377), ('doe', 0.8123292326927185), ('ani', 0.8120343685150146)]
query term :  of
query term :  environ
[('environment', 0.7685648202896118), ('develop', 0.7433871626853943), ('antipollut', 0.7183924317359924), ('internalis', 0.7170185446739197), ('ensur', 0.7157516479492188)]
query term :  axon
[('depolar', 0.9092680811882019), ('postsynapt', 0.8527619242668152), ('synapt', 0.8444193005561829), ('ganglia', 0.8421114087104797), ('innerv', 0.8401095867156982)]
query term :  termin


  2%|███▏                                                                                                                                                                                                   | 9/558 [00:01<00:48, 11.34it/s]

...

[('kind', 0.8719528317451477), ('way', 0.8681896924972534), ('doe', 0.8663068413734436), ('becaus', 0.8652352690696716), ('mean', 0.8631829619407654)]
query term :  between
query term :  a
query term :  company'
query term :  strategi
[('aim', 0.7866325378417969), ('focus', 0.7750259041786194), ('plan', 0.7727310061454773), ('effort', 0.7713293433189392), ('approach', 0.770165205001831)]
query term :  and
query term :  busi
[('firm', 0.7501569390296936), ('profit', 0.7351495027542114), ('financi', 0.7314045429229736), ('incentivis', 0.7313731908798218), ('compani', 0.7306159138679504)]
query term :  model
[('design', 0.7395058870315552), ('subconfigur', 0.7381959557533264), ('exampl', 0.7352234125137329), ('similar', 0.7203186750411987), ('ntea', 0.7200297713279724)]
query term :  is
query term :  can
query term :  fever


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 554/558 [00:18<00:00, 32.73it/s]

[('outbreak', 0.6545488238334656), ('lassa', 0.6238906383514404), ('symptom', 0.6215236783027649), ('cough', 0.6211711764335632), ('dengu', 0.6139056086540222)]
query term :  caus
[('becaus', 0.8200801014900208), ('occur', 0.812697172164917), ('sever', 0.8048741817474365), ('result', 0.7993423938751221), ('affect', 0.7967783212661743)]
query term :  miscarriag
[('stillbirth', 0.6914690136909485), ('ognal', 0.6798649430274963), ('tubal', 0.6689441800117493), ('gaudet', 0.6653184294700623), ('miscarri', 0.6573470830917358)]
query term :  earli
[('week', 0.877101480960846), ('sinc', 0.8737747073173523), ('began', 0.8693572282791138), ('befor', 0.8654952049255371), ('wa', 0.8617166876792908)]
query term :  pregnanc


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 558/558 [00:18<00:00, 30.53it/s]

[('pregnant', 0.8244889378547668), ('childbirth', 0.7837566137313843), ('fetu', 0.7825204133987427), ('ectop', 0.7590824365615845), ('prenat', 0.7572058439254761)]
dictionary ready...


In [8]:
for key in qterm_NN_dict:
    fw_NN_terms.writelines(str(key) + '\t')
    for nn in qterm_NN_dict[key]:
        fw_NN_terms.writelines(str(nn) + ',')
    fw_NN_terms.write('\n')